In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install huggingface_hub

In [ ]:
pip install transformers

In [ ]:
pip install langchain

In [ ]:
from huggingface_hub.hf_api import HfFolder

In [ ]:
HfFolder.save_token('hf_****')

In [ ]:
from transformers import BitsAndBytesConfig

In [ ]:
import torch

In [ ]:
from torch import cuda, bfloat16

In [ ]:
torch.cuda.empty_cache()

In [ ]:
torch.cuda.is_available()

In [ ]:

from huggingface_hub import login
login(token = 'hf_******')


In [ ]:
# !!! Need to accept conditions in hugging face
# import torch
# from transformers import pipeline

# model_id = "meta-llama/Llama-3.2-3B-Instruct"
# pipe = pipeline(
#     "text-generation",
#     model=model_id,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
# )
# messages = [
#     {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
#     {"role": "user", "content": "Who are you?"},
# ]
# outputs = pipe(
#     messages,
#     max_new_tokens=256,
# )
# print(outputs[0]["generated_text"][-1])

In [ ]:
pip install einops flash_attn mamba_ssm causal_conv1d

## Pure LLM's

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "infly/OpenCoder-8B-Instruct"
# infly/OpenCoder-8B-Instruct
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto",
                                             trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

messages=[
    { 'role': 'user', 'content': "write a quick sort algorithm in python."}
]

inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")

outputs = model.generate(inputs, max_new_tokens=512, do_sample=False)

result = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
print(result)

In [ ]:
messages=[
    { 'role': 'user', 'content': "Give a small summary of Ripollet, a town in Spain close to Barcelona."}
]

inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")

outputs = model.generate(inputs, max_new_tokens=512, do_sample=False)

result = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
print(result)

In [ ]:
def message_user_input(message_user='hello'):
    messages=[
        { 'role': 'user', 'content': message_user}
    ]
    
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
    
    outputs = model.generate(inputs, max_new_tokens=512, do_sample=False)
    
    result = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
    return(result)

In [ ]:
message_user_input(message_user='em pots parlar en català?')

In [ ]:
response_llm=message_user_input(message_user='em pots parlar en català?')

In [ ]:
initial_message="How can I create a videogame for children between age 5 and 10 so they can learn maths with fun?"
response_llm=message_user_input(message_user=initial_message)
print(response_llm)
for _ in range(4):

    response_llm=message_user_input(message_user=response_llm)
    print(response_llm)
    print('%%%%%%%%%%%%%%%')


## Creating a loop

# Crew AI with HuggingFace

From training:

crewai==0.28.8 -> it is giving errors, better with 0.86.0 <br>
crewai_tools==0.1.6 -> using 0.17.0<br>
langchain_community==0.0.29 -> using 0.3.13 <br>

In [ ]:
pip install crewai==0.86.0

In [ ]:
pip install crewai_tools==0.17.0

In [ ]:
pip install langchain_community==0.3.13

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables


import os
import yaml
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
os.environ["HUGGINGFACE_API_KEY"]="hf_******"

In [ ]:
# check if I can run DeepSeek


In [ ]:
# pip install vllm

In [ ]:
# vllm serve "deepseek-ai/DeepSeek-R1"

In [ ]:
# This one works for version CrewAI 0.86.0
from crewai import Agent, LLM
llm=LLM(api_key=os.getenv("HUGGINGFACE_API_KEY"),model= "huggingface/HuggingFaceH4/zephyr-7b-beta", max_tokens=6000)
#Models used:
#huggingface/HuggingFaceH4/zephyr-7b-beta -> this one works
#huggingface/Qwen/QwQ-32B-Preview -> this one is not working
#huggingface/nvidia/Llama-3.1-Nemotron-70B-Instruct-HF -> NEED TO ACCEPT AGREEMENT
#huggingface/tiiuae/Falcon3-10B-Instruct -> Not working

In [ ]:
# tried for version 0.28.0 but did not work
#from langchain_community.llms import HuggingFaceHub

#llm = HuggingFaceHub(
 #   repo_id="huggingface/HuggingFaceH4/zephyr-7b-beta",
  #  huggingfacehub_api_token="hf_*****",
   # task="text-generation"
#)

## CREW AI : Article about a topic

In [ ]:
scrape_tool = ScrapeWebsiteTool()

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    tools = [scrape_tool],
    llm=llm
)

In [ ]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

In [ ]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner
)

In [ ]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer
)

In [ ]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

In [ ]:
result = crew.kickoff(inputs={"topic": "Future of concrete"})

In [ ]:
# Checking outputs
#print(f"Raw Output: {result.raw}")
#if result.json_dict:
 #   print(f"JSON Output: {json.dumps(result.json_dict, indent=2)}")
#if result.pydantic:
   # print(f"Pydantic Output: {result.pydantic}")
#print(f"Tasks Output: {result.tasks_output}")
#print(f"Token Usage: {result.token_usage}")

In [ ]:
from IPython.display import Markdown
Markdown(result.tasks_output[-1].raw)

In [ ]:
print(f"Token Usage: {result.token_usage}")

### This is used to check token creation for a specific model

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from huggingface_hub import InferenceClient
# client=InferenceClient(api_key="hf_****")

# model_name = "HuggingFaceH4/zephyr-7b-beta"
# # infly/OpenCoder-8B-Instruct

# messages=[
#     { 'role': 'user', 'content': "what is the maximum number of tokens that you accept."}
# ]

# completion=client.chat.completions.create(model=model_name,
#                                           messages=messages,
#                                           max_tokens=500
#                                          )
# print(completion.choices[0].message)

## Crew AI: Stock analysis

In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool, FileReadTool
from crewai import Crew, Process

# search_tool = SerperDevTool() -> need tokens to connect to API
scrape_tool = ScrapeWebsiteTool()
# scrape_tool2 = ScrapeWebsiteTool(
#     website_url="https://www.expansion.com/mercados/cotizaciones/indices/ibex35_I.IB.html"
# )

filereadtool=FileReadTool()

# Agents definitions
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Check financial information of the company",
    backstory="Specializing in financial markets, this agent "
              "provides crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions. It has a lot of "
              " knowledge about Spanish stock exchange, IBEX 35. "
    "If you have errors to find the information, do not worry, you can search in https://www.expansion.com/"
    "Do not worry if you do not have all information, finish the task as soon as possible",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool], 
    llm=llm
)

# Agents definitions
data_analyst_agent_2 = Agent(
    role="Data Analyst",
    goal="Check news that could impact the company",
    backstory="This agent is able to correlate events with price movements in stocks "
    "If you have errors to find the information, do not worry, you can search in https://www.lavanguardia.com/ or https://www.expansion.com/ "
    "Do not worry if you do not have all information, finish the task as soon as possible",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool], 
    llm=llm
)

execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies. It also follows the news and other world events "
         "information to provide ideas about the evolution of the stock price. "
    "It also considers the dividends that the company pays. ",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool],
    llm=llm
)

# trading_strategy_agent = Agent(
#     role="Trading Strategy Developer",
#     goal="Develop and test various trading strategies based "
#          "on insights from the Data Analyst Agent.",
#     backstory="Equipped with a deep understanding of financial "
#               "markets and quantitative analysis, this agent "
#               "devises and refines trading strategies. It evaluates "
#               "the performance of different approaches to determine "
#               "the most profitable and risk-averse options.",
#     verbose=True,
#     allow_delegation=True,
#     # tools = [scrape_tool],
#     llm=llm
# )

risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance."
    " The agent has an invesment horizon of 1 to 3 months."
    ,
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool],
    llm=llm
)


# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Find the financial information about "
        "the selected stock ({stock_selection}) that could impact the stock price. "
        "Summarize this information for a clear view of future performance."
        "You have to include information like dividend, dividend yield, revenue, profits, costs, etc."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
       
    ),
    agent=data_analyst_agent,
)
data_analysis_task_2 = Task(
    description=(
        "Find information about "
        "news that could impact the stock price. You have to correlate them with the sector of the company {stock_selection} "
        "You have to include all current events that could impact the stock price (wars, catastrophes, terrorist attacks, economic information of Spain)."
        "Group them in positive or negative impact for the stock."
        
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
        "The information could be extracted from El Economista, Expansión or Cinco Días web pages."
    ),
    agent=data_analyst_agent_2,
)

# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Using all information provided, you have to decide if we can buy that stock or not."
        "The trading strategy will be: {trading_strategy_preference}"
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection} with the amount of {initial_capital}."
    ),
    agent=execution_agent,
)
# Task for Trading Strategy Agent: Develop Trading Strategies
# strategy_development_task = Task(
#     description=(
#         "Develop and refine trading strategies based on "
#         "the insights from the Data Analyst and "
#         "user-defined risk tolerance ({risk_tolerance}). "
#         "Consider trading preferences ({trading_strategy_preference})."
#     ),
#     expected_output=(
#         "A set of potential trading strategies for {stock_selection} "
#         "that align with the user's risk tolerance."
#     ),
#     agent=trading_strategy_agent,
# )

# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
        "The risk tolerance of the user is: {risk_tolerance}"
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)


from crewai import Crew, Process
# from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent_2,
        data_analyst_agent, 
            
            # trading_strategy_agent, 
            execution_agent,
            risk_management_agent],
    
    tasks=[data_analysis_task_2,
        data_analysis_task, 
           
           # strategy_development_task, 
        execution_planning_task,
           risk_assessment_task],
    
    manager_llm=llm,
    # process=Process.hierarchical,
    verbose=True
)

# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'International Airlines Group . Yahoo finance ticker is: IAG.MC',
    'initial_capital': '1000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Month Trading',
    'news_impact_consideration': True
}

### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

In [ ]:
result

## Crew AI: Financial analysis 2

https://learn.deeplearning.ai/courses/multi-ai-agent-systems-with-crewai/lesson/6/multi-agent-customer-support-automation-(code)

In [ ]:
pip install spider-client 'crewai[tools]'

In [ ]:
# I need an API
from crewai_tools import SpiderTool
# spider_tool = SpiderTool()

In [ ]:
from crewai_tools import SeleniumScrapingTool

In [ ]:
#Not a good tool to process tables -> for large tables, I exceed the limit of tokens and get errors
from crewai_tools import FileReadTool

# Initialize the tool to read any files the agents knows or lean the path for
# file_read_tool = FileReadTool()
filereadtool=FileReadTool(file_path="/kaggle/input/ibex-jumps-20y-stocks/sample2.csv")

In [ ]:
from crewai_tools import FileWriterTool

# Initialize the tool
file_writer_tool = FileWriterTool()

In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool, FileReadTool
from crewai import Crew, Process

# search_tool = SerperDevTool() -> need tokens to connect to API
scrape_tool = ScrapeWebsiteTool()


# filereadtool=FileReadTool(file_path="/kaggle/input/ibex-jumps-20y-stocks/sample2.csv") # takes a lot of time,
# it does not process very well excel and tables and it does not generate interesting outputs

# Agents definitions
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Identify patterns in financial assets "
         "to identify trends and predict market movements. "
    "Also compare with other stocks and compare with future commodities if necessary.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions. It has a lot of "
              " knowledge about Spanish stock exchange, IBEX 35",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool], 
    llm=llm
)

reporter_agent = Agent(
    role="Reporter agent of financial information",
    goal="Write all the information of the data analyst in a structured manner",
    backstory="Specializing in financial markets, this agent "
              "structures all financial information to be used by decision makers."
    " Also has experience in the Spanish stock exchange.",
    verbose=True,
    allow_delegation=True,
    tools = [file_writer_tool], 
    llm=llm
)

# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected company {company_selected} (yahoo finance ticker: {stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements. "
        " Your objective is the following: ({objective_agent_solve})"
        "When you finish, create a pdf to report all your findings."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

reporting_task = Task(
    description=(
        "Provide a clear view of data analyst findings."
        
    ),
    expected_output=(
        "Clear information of {stock_selection} obtained from data analyst."
    ),
    agent=reporter_agent,
)

from crewai import Crew, Process
# from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, reporter_agent],
    
    tasks=[data_analysis_task, reporting_task],
    
    # manager_llm=llm,
    # process=Process.hierarchical,
    verbose=True
)

# Example data for kicking off the process
financial_trading_inputs = {
    'company_selected':'Caixabank',
    'stock_selection': 'CABK.MC',
    'objective_agent_solve': ' I want to know the evolution of the stock in the next 2 days and compare with similar stocks'
}

### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

In [ ]:
result

# Crew AI -> Home prices: Better with OpenAI and other paying tools

In [ ]:
tool_selenium = SeleniumScrapingTool(
    website_url='https://www.idealista.com/',
    css_element='campoBus'
)

In [ ]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at Real Estate company and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
    llm=llm,
	verbose=True
)

In [ ]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at Real Estate company and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
    llm=llm
)

In [ ]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[tool_selenium],
    agent=support_agent
)

In [ ]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent
)

In [ ]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  memory=True
)

## Crew AI with Open AI

In [ ]:
files = {
    'agents': '/kaggle/input/config-crewai/agents.yaml',
    'tasks': '/kaggle/input/config-crewai/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [ ]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

In [ ]:
llm = LLM(
    model="huggingface/infly/OpenCoder-8B-Instruct",
    base_url="https://huggingface.co/v1",
    api_key="hf_*****"
)

import os
from litellm import completion

# [OPTIONAL] set env var
os.environ["HUGGINGFACE_API_KEY"] = "hf_*********"

messages = [{ "content": "There's a llama in my garden  What should I do?","role": "user"}]

# e.g. Call 'https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct' from Serverless Inference API
response = completion(
    model="huggingface/infly/OpenCoder-8B-Instruct",
    messages=[{ "content": "Hello, how are you?","role": "user"}],
    stream=True
)

print(response)

In [ ]:
# Creating Agents
project_planning_agent = Agent(llm=llm,
  config=agents_config['project_planning_agent']
)

estimation_agent = Agent(llm=llm,
  config=agents_config['estimation_agent']
)

resource_allocation_agent = Agent(llm=llm,
  config=agents_config['resource_allocation_agent']
)

# Creating Tasks
task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,
  output_pydantic=ProjectPlan # This is the structured output we want
)

# Creating Crew
crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
    
  verbose=True
)

In [ ]:
from IPython.display import display, Markdown

project = 'Website'
industry = 'Technology'
project_objectives = 'Create a website for a small business'
team_members = """
- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))

In [ ]:
# The given Python dictionary
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

## TAVILY search for information in the internet

AI Agents in LangGraph  DeepLearning.ai courses

In [ ]:
pip install langgraph

In [ ]:
pip install langchain_community

In [ ]:
pip install langchain_huggingface

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
# from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [ ]:
os.environ["TAVILY_API_KEY"] = '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'
tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

In [ ]:
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain_community.llms import HuggingFaceEndpoint
llm=HuggingFaceEndpoint(repo_id='', trust_remote_code=True)
#'nvidia/Hymba-1.5B-Instruct'
#google/flan-t5-large -> text2text, not working with that
model=ChatHuggingFace(llm=llm)

## Start with easy approach_ 
https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/2/build-an-agent-from-scratch

In [ ]:
from huggingface_hub import InferenceClient

client=InferenceClient(api_key='hf_********')

class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        inputs = tokenizer.apply_chat_template(self.messages, add_generation_prompt=True, return_tensors="pt")
        outputs = model.generate(inputs, max_new_tokens=512, do_sample=False)
        return tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)


In [ ]:
prompt = """
You are a nice assistant that answers all questions
""".strip()

def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [ ]:
abot = Agent(prompt)

In [ ]:
result = abot("How much does a toy poodle weigh?")
print(result)

# !! TO SOLVE

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [ ]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [ ]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = model  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

In [ ]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

## Image-text-to-image

In [ ]:
pip install diffusers

In [ ]:
import torch
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import make_image_grid, load_image

pipeline = AutoPipelineForImage2Image.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)

# this helps us to reduce memory usage- since SDXL is a bit heavy, this could help by
# offloading the model to CPU w/o hurting performance.
pipeline.enable_model_cpu_offload()



In [ ]:
# prepare image
url = "https://th.bing.com/th/id/R.312295bd88a0e2cf600e0d5f5733d1d9?rik=xEQTj38F2l5%2fsQ&pid=ImgRaw&r=0"

init_image = load_image(url)

prompt = "convert into a disney palette"

# pass prompt and image to pipeline
image = pipeline(prompt, image=init_image, strength=0.5).images[0]
make_image_grid([init_image, image], rows=1, cols=2)

In [ ]:
# prepare image
url = "https://www.elllobregat.com/fotos/80/escola_utmar.jpg"

init_image = load_image(url)

prompt = "convert the following image into Simpson palette"

# pass prompt and image to pipeline
image = pipeline(prompt, image=init_image, strength=0.5).images[0]
make_image_grid([init_image, image], rows=1, cols=2)

In [ ]:
# prepare image
url = "/kaggle/input/images/mariaandsara.jpg"

init_image = load_image(url)

prompt = "convert the following image into Simpson palette"

# pass prompt and image to pipeline
image = pipeline(prompt, image=init_image, strength=0.5).images[0]
make_image_grid([init_image, image], rows=1, cols=2)

In [ ]:
1+1

In [ ]:
# prepare image
url = "/kaggle/input/images/mariaandsara.jpg"

init_image = load_image(url)

prompt = "Draw a Picasso style painting"

# pass prompt and image to pipeline
image = pipeline(prompt, image=init_image, strength=0.5).images[0]
make_image_grid([init_image, image], rows=1, cols=2)